- https://automl.github.io/auto-sklearn/master/

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import pandas as pd
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split, RepeatedStratifiedKFold
import sklearn.metrics

In [3]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [4]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

In [5]:
d = pd.read_csv("data/03-sem-town-shortname.csv")
d.head()

,sales_channel_id,route_id,product_id,weight,pieces,target,state_AGUASCALIENTES,state_BAJA CALIFORNIA NORTE,state_BAJA CALIFORNIA SUR,state_CAMPECHE,...,brand_PUL,brand_RIC,brand_SAN,brand_SL,brand_SUA,brand_SUN,brand_THO,brand_TR,brand_TRI,brand_WON
0,0.0,0.203783,0.861310,0,1,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.114399,0.025384,0,1,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.124059,0.024763,0,1,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.107077,0.113427,2,2,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.107484,0.022121,2,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# get small sample of data
d = d.sample(frac=0.01)
d

,sales_channel_id,route_id,product_id,weight,pieces,target,state_AGUASCALIENTES,state_BAJA CALIFORNIA NORTE,state_BAJA CALIFORNIA SUR,state_CAMPECHE,...,brand_PUL,brand_RIC,brand_SAN,brand_SL,brand_SUA,brand_SUN,brand_THO,brand_TR,brand_TRI,brand_WON
199117,0.0,0.112772,0.024203,1,1,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102041,0.0,0.109518,0.680820,2,2,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
358763,0.0,0.122534,0.021780,2,2,1.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
914318,0.0,0.417836,0.741037,0,1,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715036,0.0,0.125686,0.725602,2,2,1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397969,0.0,0.288184,0.632274,2,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494144,0.0,0.103518,0.697676,2,2,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
872036,0.0,0.143075,0.024883,0,1,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
480104,0.0,0.449054,0.861270,1,1,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X, y = d.drop("target", axis=1), d["target"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## autosklearn
only works on ubuntu (allegedly)

In [ ]:
pip3 install autosklearn

In [ ]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

In [ ]:
scores = cross_validate(cls, X, y, cv=splitter, scoring=METRICS)
cls_scores = pd.DataFrame(scores)
pd.DataFrame(cls_scores.mean()).T

This will run for one hour and should result in an accuracy above 0.98.

In [ ]:
if __name__ == "__main__":
    X, y = sklearn.datasets.load_digits(return_X_y=True)
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier()
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

## TPOT
- http://epistasislab.github.io/tpot/api/#classification

In [8]:
from tpot import TPOTClassifier

In [ ]:
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='f1', verbosity=2, random_state=1, n_jobs=-1)
# perform the search
model.fit(X, y)
# export the best model
model.export('04 tpot_best_model.py')

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]

- tpot output on 10% of data:

Generation 1 - Current best internal CV score: 0.6992274702253357

Generation 2 - Current best internal CV score: 0.7007572509706983

Generation 3 - Current best internal CV score: 0.7038535652062866

Generation 4 - Current best internal CV score: 0.7038535652062866

Generation 5 - Current best internal CV score: 0.7038535652062866

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.1, max_depth=8, min_child_weight=16, n_estimators=100, n_jobs=1, subsample=0.8500000000000001, verbosity=0)

### optimized tpot

In [ ]:
# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, shuffle=True, random_state=1234)
# define search
model = TPOTClassifier(generations=100, population_size=100, cv=cv, scoring='f1', verbosity=2, random_state=1234, n_jobs=-1)
# perform the search
model.fit(X, y)
# export the best model
model.export('04 long-tpot_best_model.py')